# 均值交叉

In [ ]:
%pylab inline

# Pandas v0.16 之前，使用下面這個
# import pandas.io.data as web

# Pandas v0.16 之後，使用下面這個
import pandas_datareader.data as web

import pandas as pd

from pandas import Series, DataFrame

from datetime import datetime

In [ ]:
# 讀取從 1970/1/1 之後的股價資訊
df=web.DataReader("FSLR", 'yahoo', datetime(2012,1,1))

In [ ]:
# 計算均線
df['20d'] = np.round(pd.rolling_mean(df['Close'], window=20), 2)
df['60d'] = np.round(pd.rolling_mean(df['Close'], window=60), 2)

# 判斷均線向上或向下
df['20d_diff'] = np.round(df['20d'].diff(), 2)
df['60d_diff'] = np.round(df['60d'].diff(), 2)

In [ ]:
df[['Close','20d','60d']].plot(grid=True, figsize=(10,8))

## 策略

進出點的計算，可以參考：

https://www.quantstart.com/articles/Backtesting-a-Moving-Average-Crossover-in-Python-with-pandas

In [ ]:
# 第一個策略
#df['signal'] = np.where(df['20d']-df['60d'] > 0, 1.0, 0.0)

# 第二個策略
df['signal'] = np.where(np.logical_and(df['20d']-df['60d'] > 0, df['20d_diff'] > 0), 1.0, 0.0)

df['positions'] = df['signal'].diff()

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')     # Set the outer colour to white
ax1 = fig.add_subplot(111,  ylabel='Price in $')
    
# Plot the AAPL closing price overlaid with the moving averages
df['Close'].plot(ax=ax1, color='gray', lw=1., figsize=(10,8))
df[['20d', '60d']].plot(ax=ax1, lw=2., grid=True)

# Plot the "buy" trades against AAPL
ax1.plot(df.ix[df.positions == 1.0].index,df['20d'][df.positions == 1.0],'^', markersize=10, color='r')

# Plot the "sell" trades against AAPL
ax1.plot(df.ix[df.positions == -1.0].index, df['20d'][df.positions == -1.0], 'v', markersize=10, color='k')

# 如果不是在 IPython Notebook 則需要下面這一行來顯示圖形
#fig.show()

## 計算Sharpe Ratio

In [ ]:
dailyRet = df['Close'].pct_change()

In [ ]:
#假設無風險利率為 4%
#假設一年有252個交易日
excessRet = (dailyRet - 0.04/252)[df['signal']==1]

sharpeRatio = sqrt(252.0)*mean(excessRet)/std(excessRet)

In [ ]:
sharpeRatio

## 計算MaxDD跟MaxDDD

In [ ]:
# maxDD: maximum drawdown
# maxDDD: maximum drawdown duration

def calculateMaxDD(cumRet):
    highwatermark = zeros(size(cumRet))
    drawdownduration = zeros(size(cumRet))
    drawdown = zeros(size(cumRet))
    for t in range(2, cumRet.size):
        highwatermark[t] = max(highwatermark[t-1], cumRet[t])
        drawdown[t] = (1 + highwatermark[t]) / (1 + cumRet[t]) - 1
        if (drawdown[t] == 0):
            drawdownduration[t] = 0
        else:
            # 從日期來計算 MaxDDD 的天數
            # drawdownduration[t] = drawdownduration[t-1] + (cumRet.index[t]-cumRet.index[t-1]).days
            drawdownduration[t] = drawdownduration[t-1] + 1
    maxDD = max(drawdown)
    maxDDD = max(drawdownduration)
    Series(drawdownduration).plot()
    return maxDD, maxDDD

In [ ]:
cumRet = cumprod(1+excessRet) - 1

In [ ]:
calculateMaxDD(cumRet)

In [ ]:
cumRet.plot(style='ro-')